In [1]:
import pandas as pd
from sqlalchemy import create_engine
from configETL import uid, pwd, database, server

### Extract CSVs into DataFrames

In [2]:
csv_file = "movie.csv"
movie_df = pd.read_csv(csv_file)
movie_df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [3]:
csv_file = "oscars.csv"
oscars_df = pd.read_csv(csv_file)
oscars_df.head()

,Year,Ceremony,Award,Winner,Name,Film
0,1928,1,Actor,NaN,Richard Barthelmess,The Noose
1,1928,1,Actor,1.0,Emil Jannings,The Last Command
2,1928,1,Actress,NaN,Louise Dresser,A Ship Comes In
3,1928,1,Actress,1.0,Janet Gaynor,7th Heaven
4,1928,1,Actress,NaN,Gloria Swanson,Sadie Thompson


### Transform movie DataFrame

In [4]:
movie_df.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [5]:
movie1_df = movie_df[["movie_title", "title_year", "director_name", "budget", "gross", "imdb_score"]]
movie1_df.head()

,movie_title,title_year,director_name,budget,gross,imdb_score
0,Avatar,2009.0,James Cameron,237000000.0,760505847.0,7.9
1,Pirates of the Caribbean: At World's End,2007.0,Gore Verbinski,300000000.0,309404152.0,7.1
2,Spectre,2015.0,Sam Mendes,245000000.0,200074175.0,6.8
3,The Dark Knight Rises,2012.0,Christopher Nolan,250000000.0,448130642.0,8.5
4,Star Wars: Episode VII - The Force Awakens ...,NaN,Doug Walker,NaN,NaN,7.1


In [6]:
movies = movie1_df.rename(columns={"movie_title": "Film", "title_year": "Year", "director_name": "Director", "budget": "Budget", 
                               "gross": "Gross", "imdb_score": "IMDb_score"})
movies = movies[(movies.Year >= 1995) & (movies.Year <= 2015)]
movies["Budget"] = movies["Budget"].map("{:,.0f}".format)
movies["Gross"] = movies["Gross"].map("{:,.0f}".format)
movies["IMDb_score"] = movies["IMDb_score"].map("{:,}".format)
movies.head()

,Film,Year,Director,Budget,Gross,IMDb_score
0,Avatar,2009.0,James Cameron,"237,000,000","760,505,847",7.9
1,Pirates of the Caribbean: At World's End,2007.0,Gore Verbinski,"300,000,000","309,404,152",7.1
2,Spectre,2015.0,Sam Mendes,"245,000,000","200,074,175",6.8
3,The Dark Knight Rises,2012.0,Christopher Nolan,"250,000,000","448,130,642",8.5
5,John Carter,2012.0,Andrew Stanton,"263,700,000","73,058,679",6.6


In [7]:
movies.reset_index()
movies.index.name = "ID"
movies.head()

,Film,Year,Director,Budget,Gross,IMDb_score
ID,,,,,,
0,Avatar,2009.0,James Cameron,"237,000,000","760,505,847",7.9
1,Pirates of the Caribbean: At World's End,2007.0,Gore Verbinski,"300,000,000","309,404,152",7.1
2,Spectre,2015.0,Sam Mendes,"245,000,000","200,074,175",6.8
3,The Dark Knight Rises,2012.0,Christopher Nolan,"250,000,000","448,130,642",8.5
5,John Carter,2012.0,Andrew Stanton,"263,700,000","73,058,679",6.6


### Transform oscars DataFrame

In [8]:
oscars_df.columns

Index(['Year', 'Ceremony', 'Award', 'Winner', 'Name', 'Film'], dtype='object')

In [9]:
oscars_df_drop = oscars_df.drop(columns=['Ceremony', 'Winner'])
oscars_df_drop = oscars_df_drop[(oscars_df_drop.Year >= 1995) & (oscars_df_drop.Year <= 2015)]
oscars_df_drop.head()

,Year,Award,Name,Film
7228,1995,Actor in a Leading Role,Nicolas Cage,Leaving Las Vegas
7229,1995,Actor in a Leading Role,Richard Dreyfuss,Mr. Holland's Opus
7230,1995,Actor in a Leading Role,Anthony Hopkins,Nixon
7231,1995,Actor in a Leading Role,Sean Penn,Dead Man Walking
7232,1995,Actor in a Leading Role,Massimo Troisi,The Postman (Il Postino)


In [10]:
oscars = oscars_df_drop.dropna()
oscars = oscars[["Film", "Year", "Award", "Name"]]
oscars.head()

,Film,Year,Award,Name
7228,Leaving Las Vegas,1995,Actor in a Leading Role,Nicolas Cage
7229,Mr. Holland's Opus,1995,Actor in a Leading Role,Richard Dreyfuss
7230,Nixon,1995,Actor in a Leading Role,Anthony Hopkins
7231,Dead Man Walking,1995,Actor in a Leading Role,Sean Penn
7232,The Postman (Il Postino),1995,Actor in a Leading Role,Massimo Troisi


In [11]:
oscars.reset_index(inplace=True)
oscars.index.name = "ID"
oscars = oscars[["Film", "Year", "Award", "Name"]]
oscars.head()

,Film,Year,Award,Name
ID,,,,
0,Leaving Las Vegas,1995,Actor in a Leading Role,Nicolas Cage
1,Mr. Holland's Opus,1995,Actor in a Leading Role,Richard Dreyfuss
2,Nixon,1995,Actor in a Leading Role,Anthony Hopkins
3,Dead Man Walking,1995,Actor in a Leading Role,Sean Penn
4,The Postman (Il Postino),1995,Actor in a Leading Role,Massimo Troisi


### Create database connection

In [12]:
movies.dtypes

Film           object
Year          float64
Director       object
Budget         object
Gross          object
IMDb_score     object
dtype: object

In [13]:
oscars.dtypes

Film     object
Year      int64
Award    object
Name     object
dtype: object

In [14]:
engine = create_engine(f"postgres://{uid}:{pwd}@{server}/{database}")
con = engine.connect()

In [15]:
engine.table_names()

['movies', 'oscars']

### Load DataFrames into database

In [16]:
movies.to_sql(name='movies', con=engine, if_exists='append', index=True)

In [17]:
oscars.to_sql(name='oscars', con=engine, if_exists='append', index=True)